In [15]:
from langchain.agents import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from datetime import timedelta
from langchain.agents import tool, AgentExecutor
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts.chat import SystemMessagePromptTemplate
import yaml
import datetime

# Generación de agentes con langchain
Primero se definen herramientas con el decorador @tool y se agregan a una lista

In [3]:
@tool
def calcular_fechas(expresion_tiempo: str) -> list:
    """Returns a list of dates given a date expression"""
    hoy = datetime.date.today()
    fechas = []

    if expresion_tiempo == "últimos 7 días":
        for i in range(7):
            fechas.append(hoy - timedelta(days=i))
    elif expresion_tiempo == "durante el último mes":
        for i in range(30):
            fechas.append(hoy - timedelta(days=i))

    return fechas

tools = [calcular_fechas]
tools

[StructuredTool(name='calcular_fechas', description='calcular_fechas(expresion_tiempo: str) -> list - Returns a list of dates given a date expression', args_schema=<class 'pydantic.main.calcular_fechasSchemaSchema'>, func=<function calcular_fechas at 0x00000211024760C0>)]

Luego se crea un agente con un template que de las instrucciones generales y se agrega la lista de funciones al llm definido, además se pasa por un parser para que pueda pasarse a openai el parser dependerá del modelo que vaya a usar las funciones hay variois

In [13]:

def create_agent(llm, tools):
    # Crear un prompt para guiar al modelo
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Eres un asistente que ayuda a calcular rangos de fechas. Tu tarea es devolver una lista de fechas basado en los resultados del agente. no es necesaria ninguna explicación, sólo la lista en formato de python tu respuesta debe ser como este ejemplo con las fechas correspondientes: ['2023-01-01', '2023-02-01']. El resultado del agente es:"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    # Configuración para pasar las herramientas al agente
    llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

    # Crear el agente utilizando los componentes de LangChain
    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
        }
        | prompt
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser()
    )

    return agent

Se ejecuta el agente

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
AgentExecutor(agent=create_agent(llm, tools), tools=tools, verbose=True).invoke({"input": 'últimos 7 días'})



> Entering new AgentExecutor chain...

Invoking: `calcular_fechas` with `{'expresion_tiempo': 'últimos 7 días'}`


[datetime.date(2023, 12, 20), datetime.date(2023, 12, 19), datetime.date(2023, 12, 18), datetime.date(2023, 12, 17), datetime.date(2023, 12, 16), datetime.date(2023, 12, 15), datetime.date(2023, 12, 14)]['2023-12-20', '2023-12-19', '2023-12-18', '2023-12-17', '2023-12-16', '2023-12-15', '2023-12-14']

> Finished chain.


{'input': 'últimos 7 días',
 'output': "['2023-12-20', '2023-12-19', '2023-12-18', '2023-12-17', '2023-12-16', '2023-12-15', '2023-12-14']"}

In [16]:
def get_date_list(prompt):
    system_message_prompt = SystemMessagePromptTemplate.from_template(yaml.safe_load(open("templates.yaml"))['DATES'])
    template = ChatPromptTemplate.from_messages([system_message_prompt])
    model = ChatOpenAI(model="gpt-3.5-turbo")

    chain = (
        {"question": RunnablePassthrough(), "today": RunnablePassthrough()} 
        | template
        | model
        | StrOutputParser()
    )

    hoy = datetime.date.today()
    result = chain.invoke({"question": f"{prompt}", "today": f"{hoy}"})
    return result

In [17]:
get_date_list('dame los comentarios de hace un mes')

"['2023-11-20', '2023-11-21', '2023-11-22', '2023-11-23', '2023-11-24', '2023-11-25', '2023-11-26', '2023-11-27', '2023-11-28', '2023-11-29', '2023-11-30', '2023-12-01', '2023-12-02', '2023-12-03', '2023-12-04', '2023-12-05', '2023-12-06', '2023-12-07', '2023-12-08', '2023-12-09', '2023-12-10', '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15', '2023-12-16', '2023-12-17', '2023-12-18', '2023-12-19']"